In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Load names of classes, contains 80 diff classes
classesFile = "coco.names";
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

In [3]:
# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "yolov3.cfg";
modelWeights = "yolov3.weights";

In [4]:
# read pre-trained model and config file
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)

In [5]:
def get_imageDetect(input_image, image):
    # create input blob 
    # set input blob for the network 
    net.setInput(cv2.dnn.blobFromImage(input_image, 0.00392, (416,416), (0,0,0), True, crop=False))

    # run inference through the network
    # and gather predictions from output layers

    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    outs = net.forward(output_layers)



    Width = input_image.shape[1]
    Height = input_image.shape[0]

    class_ids = []
    confidences = []
    boxes = []

    #create bounding box 
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.1:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)

    #check if is people detection
    for i in indices:
        i = i[0]
        box = boxes[i]
        if class_ids[i]==0:
            label = str(classes[class_id]) 
            cv2.rectangle(input_image, (round(box[0]),round(box[1])), (round(box[0]+box[2]),round(box[1]+box[3])), (0, 0, 0), 2)
            cv2.putText(input_image, label, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

#     plt.imshow(input_image)  
#     cv2.imwrite(str(image[0:-6])+'_op_detect.jpg', input_image)


In [6]:
image_list = [image for image in glob.glob('291_dataSet_new/*.jpg')]


In [7]:
for image in image_list:
    input_image = cv2.imread(image)
    get_imageDetect(input_image, image)